In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df = pd.read_csv("https://data.wprdc.org/datastore/dump/e03a89dd-134a-4ee8-a2bd-62c40aeebc6f")

In [3]:
def combine_names(x):
    if 'Lawrenceville' in x:
        x = 'Lawrenceville'
    elif 'Oakland' in x:
        x = 'Oakland'
    elif 'Squirrel Hill' in x:
        x = 'Squirrel Hill'
    return x

district_names = ["Central Business District","Strip District","Shadyside","Squirrel Hill",
                  "Oakland","South Side Flats","Bloomfield","Highland Park","East Liberty",
                  "North Shore","Lawrenceville","Greenfield","Hazelwood"]

df.columns = df.columns.str.lower()
df = df[['_id','age','gender','race','incidentneighborhood']]
df = df.dropna()
df['incidentneighborhood'] = df['incidentneighborhood'].apply(combine_names)
df = df[df['incidentneighborhood'].isin(district_names)]


df

,_id,age,gender,race,incidentneighborhood
0,1,42.0,F,B,Bloomfield
5,6,45.0,M,W,Greenfield
8,9,17.0,M,B,East Liberty
17,18,23.0,M,B,South Side Flats
20,21,41.0,M,B,South Side Flats
...,...,...,...,...,...
66466,88204,50.0,M,B,Central Business District
66467,88205,18.0,M,U,Shadyside
66472,88268,47.0,M,B,Central Business District
66473,88269,41.0,M,W,South Side Flats
